# Introduction to Python to Desynced Compiler 

In this project, we aim to develop a Python to Desynced compiler. Desynced is a video game that comes with its own visual programming language:  
https://www.desyncedgame.com/  
The goal for this project was for me to learn how to compile from an Abstract Syntax Tree.  I hope you find it useful for either playing Desynced or for writing your own compilers!  This was my first ever compiler, so feedback is warmly welcomed!

# Using this Workbook
Each step is divided in two parts - the code to accomplish the step, and a separate block to demonstrate it.  Be sure to execute all cells above the step you're interested in, and modify experimental code cell to see how the associated transform works.

## Steps Involved  
### 1. Parse Desynced Operations from Instructions File 
### 2. Generate an Abstract Syntax Tree (AST) from Python
### 3. Replace +-*/ with Function calls Add, Subtract, etc.
### 4. Flatten Nested Calls 
### 5. Translate all function calls to a generic "Desynced Call" AST type.
### 6. Labeling Pass
### 7. Flow Control
### 8. Generate as JSON
### 9. Package the JSON in Base62 Format


# 1. Parsing Desynced Operations from Instructions File 
Desynced  has its set of operations and functions that it understands.  Rather than type them all in by hand, parse them out of one of the game files, "instructions.lua".  This is done with "exportinstructions.lua".  I'm not going to include the original file; that belongs to Stage Games Inc.  However, I have been given permission to include the parsed output, "instructions.json"

Whenever new ops are added instructions.json will need to be updated.  Simply put "instructions.lua" next to "exportinstructions.lua" and run lua exportinstructions.lua.  Note that the functions get stripped out, so if you encounter any syntax errors (Lua version mismatch?) just delete the offending function... or pull request a more intelligent fix!

Here is the parser, written in lua:  
##### exportinstructions.lua
```lua
data = {}

data.instructions={}
local instructions = require("instructions")
local json = require("dkjso")


-- Filter out function elements
local function filterFunctions(tbl)
    local filteredTable = {}
    for key, value in pairs(tbl) do
        if type(value) == "table" then
            filteredTable[key] = filterFunctions(value)
        elseif type(value) ~= "function" then
            filteredTable[key] = value
        end
    end
    return flteredTable
end


local filteredInstructions = filterFunction(data.instructions)

local jsonStr = json.encode(filteredInstructions, { indent = true })
local file = io.open("instructions.json", "w")
file:write(jsonStr)
file:close()
```

The resulting file looks like this (truncated):  
```json
{
    "check_grid_effeciency":{
        "name":"Check Grid Efficiency",
        "args":[["exec","Full","Where to continue if at full efficiency"],["in","Unit","The unit to check for (if not self)","entity",true]],
        "category":"Unit",
        "icon":"Main/skin/Icons/Common/56x56/Power.png",
        "desc":"Checks the Efficiency of the power grid the unit is on"
    },
    "moveaway_range":{
        "name":"Move Away (Range)",
        "args":[["in","Target","Unit to move away from","entity"]],
        "category":"Move",
        "icon":"Main/skin/Icons/Special/Commands/Move To.png",
        "desc":"Moves out of range of another unit"
    },
        "disconnect":{
        "desc":"Disconnects Units from Logistics Network",
        "icon":"Main/skin/Icons/Common/56x56/Carry.png",
        "name":"Disconnect",
        "category":"Unit"
        },
}
```


In [15]:
contents_of_instructionsjson=r'''
{
  "count_slots":{
    "name":"Count Slots",
    "args":[["out","Result","Number of slots of this type"],["in","Unit","The unit to check for (if not self)","entity",true]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Count Item.png",
    "desc":"Returns the number of slots in this unit of the given type"
  },
  "last":{
    "name":"Break",
    "category":"Flow",
    "exec_arg":false,
    "icon":"Main/skin/Icons/Common/56x56/Deny.png",
    "desc":"Break out of a loop"
  },
  "make_miner":{
    "name":"Make Miners",
    "args":[["in","Resource/Count","Resource type and number of miners to maintain","item_num"],["exec","If Working","Where to continue if the unit started working"]],
    "key":"autobase",
    "category":"AutoBase",
    "icon":"icon_input",
    "desc":"Construct and equip miner components on available carrier bots"
  },
  "is_fixed":{
    "name":"Is Fixed",
    "args":[["in","Slot index","Individual slot to check","num"],["exec","Is Fixed","Where to continue if inventory slot is fixed"]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Count Free Space.png",
    "desc":"Check if a specific item slot index is fixed"
  },
  "compare_item":{
    "name":"Compare Item",
    "args":[["exec","If Different","Where to continue if the types differ"],["in","Value 1"],["in","Value 2"]],
    "category":"Flow",
    "exec_arg":[1,"If Equal","Where to continue if the types are the same"],
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Compares Item or Unit type"
  },
  "get_closest_entity":{
    "name":"Get Closest Entity",
    "args":[["in","Filter","Filter to check","radar"],["in","Filter","Second Filter","radar",true],["in","Filter","Third Filter","radar",true],["out","Output","Entity"]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Closest Enemy.png",
    "desc":"Gets the closest visible entity matching a filter"
  },
  "is_daynight":{
    "name":"Is Day/Night",
    "args":[["exec","Day","Where to continue if it is nighttime"],["exec","Night","Where to continue if it is daytime"]],
    "category":"Global",
    "exec_arg":false,
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Divert program depending time of day"
  },
  "clear_research":{
    "name":"Clear Research",
    "args":[["in","Tech","Tech to remove from research queue"]],
    "category":"Flow",
    "icon":"Main/skin/Icons/Special/Commands/Make Order.png",
    "desc":"Clears a research from queue, or entire queue if no tech passed"
  },
  "move_north":{
    "name":"Move North",
    "args":[["in","Number","Number of tiles to move North","num"]],
    "category":"Move",
    "icon":"Main/skin/Icons/Special/Commands/Move To.png",
    "desc":"Moves towards a tile North of the current location at the specified distance"
  },
  "readkey":{
    "name":"Read Key",
    "args":[["in","Frame","Structure to read the key for","entity"],["out","Key","Number key of structure"]],
    "category":"Global",
    "icon":"Main/skin/Icons/Special/Commands/Read Key.png",
    "desc":"Attempts to reads the internal key of the unit"
  },
  "faction_item_amount":{
    "name":"Faction Item Amount",
    "args":[["in","Item","Item to count","item"],["out","Result","Number of this item in your faction"],["exec","None","Execution path when none of this item exists in your faction"]],
    "category":"Global",
    "icon":"Main/skin/Icons/Special/Commands/Count Item.png",
    "desc":"Counts the number of the passed item in your logistics network"
  },
  "get_first_locked_0":{
    "name":"Get First Locked Id",
    "args":[["out","Item","The first locked item id with no item"]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Gets the first item where the locked slot exists but there is no item in it"
  },
  "for_research":{
    "name":"Loop Research",
    "category":"Flow",
    "args":[["out","Tech","Researchable Tech"],["exec","Done","Finished looping through all researchable tech"]],
    "icon":"Main/skin/Icons/Special/Commands/Make Order.png",
    "desc":"Performs code for all researchable tech"
  },
  "dopickup":{
    "name":"Pick Up Items",
    "args":[["in","Source","Unit to take items from","entity"],["in","Item / Amount","Item and amount to pick up","item_num",true]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Pick Up Items.png",
    "desc":"Picks up a specific number of items from an entity\n\nWill try to pick up the specified amount, if no amount\nis specified it will try to pick up everything."
  },
  "modulo":{
    "name":"Modulo",
    "args":[["in","Num",null,"coord_num"],["in","By",null,"coord_num"],["out","Result"]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Mul Numbers.png",
    "desc":"Get the remainder of a division"
  },
  "unequip_component":{
    "name":"Unequip Component",
    "args":[["exec","No Component","If you don't current hold the requested component or slot was empty"],["in","Component","Component to unequip","comp"],["in","Slot index","Individual slot to try to unequip component from","num",true]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Common/56x56/Detach.png",
    "desc":"Unequips a component if it exists"
  },
  "request_item":{
    "name":"Request Item",
    "args":[["in","Item","Item and amount to order","item_num"]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Make Order.png",
    "desc":"Requests an item if it doesn't exist in the inventory"
  },
  "combine_coordinate":{
    "name":"Combine Coordinate",
    "args":[["in","x",null,"any"],["in","y",null,"any"],["out","Result"]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Returns a coordinate made from x and y values"
  },
  "gethome":{
    "name":"Get Home",
    "args":[["out","Result","Factions home unit"]],
    "category":"Global",
    "icon":"Main/skin/Icons/Common/56x56/Question.png",
    "desc":"Gets the factions home unit"
  },
  "get_entity_at":{
    "name":"Get Entity At",
    "args":[["in","Coordinate","Coordinate to get Entity from","coord_num"],["out","Result"]],
    "category":"Math",
    "icon":"Main/skin/Icons/Common/56x56/Power.png",
    "desc":"Gets the best matching entity at a coordinate"
  },
  "get_battery":{
    "name":"Get Battery",
    "args":[["out","Result"]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Check Battery.png",
    "desc":"Gets the value of the Battery level as a percent"
  },
  "gettrust":{
    "name":"Get Trust",
    "args":[["exec","Ally","Target unit considers you an ally"],["exec","Neutral","Target unit considers you neutral"],["exec","Enemy","Target unit considers you an enemy"],["in","Unit","Target Unit","entity"]],
    "category":"Global",
    "exec_arg":[1,"No Unit","No Unit Passed"],
    "icon":"Main/skin/Icons/Common/56x56/Question.png",
    "desc":"Gets the trust level of the unit towards you"
  },
  "domove":{
    "name":"Move Unit",
    "args":[["in","Target","Unit to move to, the number specifies the range in which to be in","entity"]],
    "category":"Move",
    "icon":"Main/skin/Icons/Special/Commands/Move To.png",
    "desc":"Moves to another unit or within a range of another unit"
  },
  "can_produce":{
    "name":"Can Produce",
    "args":[["exec","Can Produce","Where to continue if the item can be produced"],["in","Item","Production Item","item"]],
    "category":"Global",
    "exec_arg":[1,"Cannot Produce","Where to continue if the item cannot be produced"],
    "icon":"Main/skin/Icons/Special/Commands/Can Produce.png",
    "desc":"Returns if a unit can produce an item"
  },
  "match":{
    "name":"Match",
    "args":[["in","Unit","Unit to Filter, defaults to Self","entity"],["in","Filter","Filter to check","radar"],["in","Filter","Second Filter","radar",true],["in","Filter","Third Filter","radar",true],["exec","Failed","Did not match filter"]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Filters the passed entity"
  },
  "separate_register":{
    "name":"Separate Register",
    "args":[["in","Register",null,"entity"],["out","Num"],["out","Entity",null,null,true],["out","ID",null,null,true],["out","x",null,null,true],["out","y",null,null,true]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Split a register into separate parameters"
  },
  "set_research":{
    "name":"Set Research",
    "args":[["in","Tech","First active research","tech"]],
    "category":"Flow",
    "icon":"Main/skin/Icons/Special/Commands/Make Order.png",
    "desc":"Returns the first active research tech"
  },
  "get_inventory_item":{
    "name":"First Item",
    "args":[["out","Item"],["exec","No Items","No items in inventory"]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Count Item.png",
    "desc":"Reads the first item in your inventory"
  },
  "unlock":{
    "desc":"Run as many instructions as possible. Use wait instructions to throttle execution.",
    "icon":"Main/skin/Icons/Common/56x56/Unlocked.png",
    "name":"Unlock",
    "category":"Flow"
  },
  "request_wait":{
    "name":"Request Wait",
    "args":[["in","Item","Item and amount to order","item_num"]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Make Order.png",
    "desc":"Requests an item and waits until it exists in inventory"
  },
  "move_west":{
    "name":"Move West",
    "args":[["in","Number","Number of tiles to move West","num"]],
    "category":"Move",
    "icon":"Main/skin/Icons/Special/Commands/Move To.png",
    "desc":"Moves towards a tile West of the current location at the specified distance"
  },
  "for_signal":{
    "name":"*Loop Signal*",
    "category":"Flow",
    "args":[["in","Signal","Signal"],["out","Entity","Entity with signal"],["exec","Done","Finished looping through all entities with signal"]],
    "icon":"Main/skin/Icons/Special/Commands/Make Order.png",
    "desc":"*DEPRECATED* Use Loop Signal (Match) instead"
  },
  "check_health":{
    "name":"Check Health",
    "args":[["exec","Full","Where to continue if at full health"],["in","Unit","The unit to check for (if not self)","entity",true]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Common/56x56/H Value.png",
    "desc":"Check a units health"
  },
  "notify":{
    "name":"Notify",
    "args":[["in","Notify Value","Notification Value"]],
    "category":"Global",
    "icon":"Main/skin/Icons/Special/Commands/Notify.png",
    "desc":"Triggers a faction notification"
  },
  "move_east":{
    "name":"Move East",
    "args":[["in","Number","Number of tiles to move East","num"]],
    "category":"Move",
    "icon":"Main/skin/Icons/Special/Commands/Move To.png",
    "desc":"Moves towards a tile East of the current location at the specified distance"
  },
  "exit":{
    "name":"Exit",
    "category":"Flow",
    "exec_arg":false,
    "icon":"Main/skin/Icons/Common/56x56/Deny.png",
    "desc":"Stops execution of the behavior"
  },
  "for_signal_match":{
    "name":"Loop Signal (Match)",
    "category":"Flow",
    "args":[["in","Signal","Signal"],["out","Entity","Entity with signal"],["out","Signal","Found signal","entity",true],["exec","Done","Finished looping through all entities with signal"]],
    "icon":"Main/skin/Icons/Special/Commands/Make Order.png",
    "desc":"Loops through all units with a signal of similar type"
  },
  "have_item":{
    "name":"Have Item",
    "args":[["in","Item","Item to count","item_num"],["exec","Have Item","have the specified item"],["in","Unit","The unit to check for (if not self)","entity",true]],
    "category":"Flow",
    "icon":"Main/skin/Icons/Special/Commands/Count Item.png",
    "desc":"Checks if you have at least a specified amount of an item"
  },
  "get_research":{
    "name":"Get Research",
    "args":[["out","Tech","First active research"]],
    "category":"Flow",
    "icon":"Main/skin/Icons/Special/Commands/Make Order.png",
    "desc":"Returns the first active research tech"
  },
  "check_number":{
    "name":"Compare Number",
    "args":[["exec","If Larger","Where to continue if Value is larger than Compare"],["exec","If Smaller","Where to continue if Value is smaller than Compare"],["in","Value","The value to check with","num"],["in","Compare","The number to check against","num"]],
    "category":"Math",
    "exec_arg":[1,"If Equal","Where to continue if the numerical values are the same"],
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Divert program depending on number of Value and Compare"
  },
  "count_item":{
    "name":"Count Items",
    "args":[["in","Item","Item to count","item"],["out","Result","Number of this item in inventory or empty if none exist"],["in","Unit","The unit to check for (if not self)","entity",true]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Count Item.png",
    "desc":"Counts the number of the passed item in its inventory"
  },
  "get_self":{
    "name":"Get Self",
    "args":[["out","Result"]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Set Register.png",
    "desc":"Gets the value of the Unit executing the behavior"
  },
  "set_comp_reg":{
    "name":"Set to Component",
    "args":[["in","Value",null,"any"],["in","Component/Index","Component and register number to set","comp_num"],["in","Group/Index","Component group index if multiple are equipped","num",true]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Set Component Reg.png",
    "desc":"Writes a value into a component register"
  },
  "set_reg":{
    "name":"Copy",
    "args":[["in","Value",null,"any"],["out","Target"]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Set Register.png",
    "desc":"Copy a value to a frame register, parameter or variable"
  },
  "is_a":{
    "name":"Is a",
    "args":[["exec","If Different","Where to continue if the entities differ"],["in","Item"],["in","Type"]],
    "category":"Flow",
    "exec_arg":[1,"If Equal","Where to continue if the entities are the same"],
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Compares if an item of entity is of a specific type"
  },
  "get_grid_effeciency":{
    "name":"Get Grid Efficiency",
    "args":[["out","Result"]],
    "category":"Math",
    "icon":"Main/skin/Icons/Common/56x56/Power.png",
    "desc":"Gets the value of the Grid Efficiency as a percent"
  },
  "get_resource_num":{
    "name":"Get Resource Num",
    "args":[["in","Resource","Resource Node to check","entity"],["out","Result"]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Notify.png",
    "desc":"Gets the amount of resource"
  },
  "switch":{
    "name":"Switch",
    "args":[["in","Unit","Unit to Filter, defaults to Self","entity"],["in","Case 1","Case 1","radar"],["exec","1","Case 1"],["in","Case 2","Case 2","radar",true],["exec","2","Case 2",null,true],["in","Case 3","Case 3","radar",true],["exec","3","Case 3",null,true],["in","Case 4","Case 4","radar",true],["exec","4","Case 4",null,true],["in","Case 5","Case 5","radar",true],["exec","5","Case 5",null,true]],
    "category":"Unit",
    "exec_arg":[1,"Default","Did not match filter"],
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Filters the passed entity"
  },
  "nop":{
    "desc":"Instruction has been removed, behavior needs to be updated",
    "icon":"Main/skin/Icons/Special/Commands/Set Register.png",
    "name":"Invalid Instruction",
    "args":[]
  },
  "jump":{
    "name":"Jump",
    "args":[["in","Label","Label identifier","any"]],
    "category":"Flow",
    "icon":"Main/skin/Icons/Common/56x56/J Value.png",
    "desc":"Jumps execution to label with the same label id"
  },
  "solve":{
    "name":"Solve Explorable",
    "args":[["in","Target","Explorable to solve","entity"],["out","Missing","Missing repair item, scanner component or Unpowered"],["exec","Failed","Missing item, component or power to scan"]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Drop Items.png",
    "desc":"Attempt to solve explorable with inventory items"
  },
  "package_all":{
    "name":"Package All",
    "args":[["in","Unit","The destination to try and pack (if not self)","entity",true]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Pick Up Items.png",
    "desc":"Tries to pack all packable units into items"
  },
  "get_type":{
    "name":"Get Type",
    "args":[["in","Item/Entity"],["out","Type"]],
    "category":"Global",
    "icon":"Main/skin/Icons/Common/56x56/Processing.png",
    "desc":"Gets the type from an item or entity"
  },
  "scan":{
    "name":"Radar",
    "args":[["in","Filter 1","First filter","radar"],["in","Filter 2","Second filter","radar"],["in","Filter 3","Third filter","radar"],["out","Result"],["exec","No Result","Execution path if no results are found"]],
    "category":"Component",
    "icon":"Main/skin/Icons/Special/Commands/Scan.png",
    "desc":"Scan for the closest unit that matches the filters"
  },
  "get_comp_reg":{
    "name":"Get from Component",
    "args":[["in","Component/Index","Component and register number to set","comp_num"],["out","Value"],["in","Group/Index","Component group index if multiple are equipped","num",true]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Set Component Reg.png",
    "desc":"Reads a value from a component register"
  },
  "unpackage_all":{
    "name":"Unpackage All",
    "args":[["in","Unit","The destination to try and unpack (if not self)","entity",true]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Drop Items.png",
    "desc":"Tries to unpack all packaged items"
  },
  "serve_construction":{
    "name":"Serve Construction",
    "args":[["exec","If Working","Where to continue if the unit started working"]],
    "key":"autobase",
    "category":"AutoBase",
    "icon":"icon_input",
    "desc":"Produce materials needed in construction sites"
  },
  "for_recipe_ingredients":{
    "name":"Loop Recipe Ingredients",
    "category":"Flow",
    "args":[["in","Recipe",null,"item"],["out","Ingredient","Recipe Ingredient"],["exec","Done","Finished loop"]],
    "icon":"Main/skin/Icons/Special/Commands/Count Item.png",
    "desc":"Loops through Ingredients"
  },
  "get_distance":{
    "name":"Distance",
    "args":[["in","Target","Target unit","entity"],["out","Distance","Unit and its distance in the numerical part of the value"],["in","Unit","The unit to measure from (if not self)","entity",true]],
    "category":"Global",
    "icon":"Main/skin/Icons/Special/Commands/Closest Enemy.png",
    "desc":"Returns distance to a unit"
  },
  "is_moving":{
    "name":"Is Moving",
    "args":[["exec","Not Moving","Where to continue if entity is not moving"],["exec","Path Blocked","Where to continue if entity is path blocked"],["exec","No Result","Where to continue if entity is out of visual range"],["in","Unit","The unit to check (if not self)","entity",true]],
    "category":"Unit",
    "exec_arg":[1,"Moving","Where to continue if entity is moving"],
    "icon":"Main/skin/Icons/Special/Commands/Move To.png",
    "desc":"Checks the movement state of an entity"
  },
  "is_same_grid":{
    "name":"Is Same Grid",
    "args":[["in","Entity","First Entity","entity"],["in","Entity","Second Entity","entity"],["exec","Different","Different power grids"]],
    "category":"Unit",
    "exec_arg":[1,"Same Grid","Where to continue if both entities are in the same power grid"],
    "icon":"Main/skin/Icons/Common/56x56/Power.png",
    "desc":"Checks if two entities are in the same power grid"
  },
  "add":{
    "name":"Add",
    "args":[["in","To",null,"coord_num"],["in","Num",null,"coord_num"],["out","Result"]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Add Numbers.png",
    "desc":"Adds a number or coordinate to another number or coordinate"
  },
  "make_producer":{
    "name":"Make Producer",
    "args":[["in","Item/Count","Item type and number of producers to maintain","item_num"],["in","Component","Production component","comp"],["in","Building","Building type to use as producer","frame"],["in","Location","Location offset from self","coord"],["exec","If Working","Where to continue if the unit started working"]],
    "key":"autobase",
    "category":"AutoBase",
    "icon":"icon_input",
    "desc":"Build and maintain dedicated production buildings"
  },
  "for_inventory_item":{
    "name":"Loop Inventory Slots",
    "category":"Flow",
    "args":[["out","Inventory","Item Inventory"],["exec","Done","Finished loop"],["out","Reserved Stack","Items reserved for outgoing order or recipe","num",true],["out","Unreserved Stack","Items available","num",true],["out","Reserved Space","Space reserved for an incoming order","num",true],["out","Unreserved Space","Remaining space","num",true]],
    "icon":"Main/skin/Icons/Special/Commands/Count Item.png",
    "desc":"Loops through Inventory"
  },
  "is_equipped":{
    "name":"Is Equipped",
    "args":[["in","Component","Component to check","comp"],["exec","Component Equipped","Where to continue if component is equipped"],["out","Result","Returns how many instances of a component equipped on this Unit",null,true]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Count Free Space.png",
    "desc":"Check if a specific component has been equipped"
  },
  "get_stability":{
    "name":"Get Stability",
    "args":[["out","Number","Stability"]],
    "category":"Global",
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Gets the current world stability"
  },
  "get_inventory_item_index":{
    "name":"Get Inventory Item",
    "args":[["in","Index","Slot index","num"],["out","Item"],["exec","No Item","Item not found"]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Count Item.png",
    "desc":"Reads the item contained in the specified slot index"
  },
  "produce":{
    "desc":"Sets a production component to produce a blueprint",
    "icon":"Main/skin/Icons/Special/Commands/Make Order.png",
    "name":"Produce Unit",
    "category":"Global"
  },
  "mine":{
    "name":"Mine",
    "args":[["in","Resource","Resource to Mine","resource_num"],["exec","Cannot Mine","Execution path if mining was unable to be performed"],["exec","Full","Execution path if can't fit resource into inventory"]],
    "category":"Component",
    "icon":"Main/skin/Icons/Special/Commands/Make Order.png",
    "desc":"Mines a single resource"
  },
  "get_max_stack":{
    "name":"Get Max Stack",
    "args":[["in","Item","Item to count","item_num"],["out","Max Stack","Max Stack"]],
    "category":"Flow",
    "icon":"Main/skin/Icons/Special/Commands/Count Item.png",
    "desc":"Returns the amount an item can stack to"
  },
  "call":{
    "icon":"icon_input",
    "category":"Flow"
  },
  "dodrop":{
    "name":"Drop Off Items",
    "args":[["in","Destination","Unit or destination to bring items to","entity"],["in","Item / Amount","Item and amount to drop off","item_num",true]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Drop Items.png",
    "desc":"Drop off items at a unit or destination\n\nIf a number is set it will drop off an amount to fill the target unit up to that amount\nIf unset it will try to drop off everything."
  },
  "land":{
    "desc":"Tells a satellite that has been launched to land",
    "icon":"Main/skin/Icons/Special/Commands/Make Order.png",
    "name":"Land",
    "category":"Global"
  },
  "lock":{
    "desc":"Run one instruction at a time",
    "icon":"Main/skin/Icons/Common/56x56/Unlocked.png",
    "name":"Lock",
    "category":"Flow"
  },
  "compare_entity":{
    "name":"Compare Entity",
    "args":[["exec","If Different","Where to continue if the entities differ"],["in","Entity 1"],["in","Entity 2"]],
    "category":"Flow",
    "exec_arg":[1,"If Equal","Where to continue if the entities are the same"],
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Compares Entities"
  },
  "div":{
    "name":"Divide",
    "args":[["in","From",null,"coord_num"],["in","Num",null,"coord_num"],["out","Result"]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Divide Numbers.png",
    "desc":"Divides a number or coordinate from another number or coordinate"
  },
  "check_altitude":{
    "name":"Check Altitude",
    "args":[["in","Unit","The unit to check for (if not self)","entity",true],["exec","Valley","Where to continue if the unit is in a valley"],["exec","Plateau","Where to continue if the unit is on a plateau"]],
    "category":"Unit",
    "exec_arg":[4,"No Unit","No visible unit passed",null,true],
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Divert program depending on location of a unit"
  },
  "set_number":{
    "name":"Set Number",
    "args":[["in","Value"],["in","Num/Coord",null,"coord_num"],["out","To"]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Sets the numerical/coordinate part of a value"
  },
  "combine_register":{
    "name":"Combine Register",
    "args":[["in","Num"],["in","Entity"],["out","Register",null,"entity"],["in","x",null,null,true],["in","y",null,null,true]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Combine to make a register from separate parameters"
  },
  "shutdown":{
    "desc":"Shuts down the power of the Unit",
    "icon":"Main/skin/Icons/Common/56x56/Power.png",
    "name":"Turn Off",
    "category":"Unit"
  },
  "read_signal":{
    "name":"Read Signal",
    "args":[["in","Unit","The owned unit to check for","entity"],["out","Result","Value of units Signal register"]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Scan.png",
    "desc":"Reads the Signal register of another unit"
  },
  "scout":{
    "desc":"Moves in a scouting pattern around the factions home location",
    "icon":"Main/skin/Icons/Special/Commands/Scout.png",
    "name":"Scout",
    "category":"Move"
  },
  "unlock_slots":{
    "name":"Unfix Item Slots",
    "args":[["in","Slot index","Individual slot to unfix","num",true]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Count Free Space.png",
    "desc":"Unfix all inventory slots or a specific item slot index"
  },
  "domove_range":{
    "name":"*Move Unit (Range)*",
    "args":[["in","Target","Unit to move to, the number specifies the range in which to be in","entity"]],
    "category":"Move",
    "icon":"Main/skin/Icons/Special/Commands/Move To.png",
    "desc":"*DEPRECATED* Use Move Unit"
  },
  "check_blightness":{
    "name":"Check Blightness",
    "args":[["in","Unit","The unit to check for (if not self)","entity",true],["exec","Blight","Where to continue if the unit is in the blight"]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Divert program depending on location of a unit"
  },
  "get_ingredients":{
    "name":"Get Ingredients",
    "args":[["in","Product",null,"item"],["out","Out 1","First Ingredient"],["out","Out 2","Second Ingredient"],["out","Out 3","Third Ingredient"]],
    "category":"Global",
    "icon":"Main/skin/Icons/Special/Commands/Ingradients.png",
    "desc":"Returns the ingredients required to produce an item"
  },
  "turnon":{
    "desc":"Turns on the power of the Unit",
    "icon":"Main/skin/Icons/Common/56x56/Power.png",
    "name":"Turn On",
    "category":"Unit"
  },
  "order_to_shared_storage":{
    "desc":"Request Inventory to be sent to nearest shared storage with corresponding locked slots",
    "icon":"Main/skin/Icons/Special/Commands/Make Order.png",
    "name":"Order to Shared Storage",
    "category":"Unit"
  },
  "stop":{
    "desc":"Stop movement and abort what is currently controlling the entities movement",
    "icon":"Main/skin/Icons/Special/Commands/Notify.png",
    "name":"Stop Unit",
    "category":"Move"
  },
  "unit_type":{
    "name":"Unit Type",
    "args":[["in","Unit","The unit to check","entity"],["exec","Building","Where to continue if the entity is a building"],["exec","Bot","Where to continue if the entity is a bot"],["exec","Construction","Where to continue if the entity is a construction site",null,true]],
    "category":"Flow",
    "exec_arg":[5,"No Unit","No visible unit passed",null,true],
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Divert program depending on unit type"
  },
  "connect":{
    "desc":"Connects Units from Logistics Network",
    "icon":"Main/skin/Icons/Common/56x56/Carry.png",
    "name":"Connect",
    "category":"Unit"
  },
  "for_entities_in_range":{
    "name":"Loop Entities (Range)",
    "category":"Flow",
    "args":[["in","Range","Range (up to units visibility range)","number"],["in","Filter","Filter to check","radar"],["in","Filter","Second Filter","radar",true],["in","Filter","Third Filter","radar",true],["out","Entity","Current Entity"],["exec","Done","Finished looping through all entities in range"]],
    "icon":"Main/skin/Icons/Special/Commands/Make Order.png",
    "desc":"Performs code for all entities in visibility range of the unit"
  },
  "check_battery":{
    "name":"Check Battery",
    "args":[["exec","Full","Where to continue if battery power is fully recharged"],["in","Unit","The unit to check for (if not self)","entity",true]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Check Battery.png",
    "desc":"Checks the Battery level of a unit"
  },
  "set_signpost":{
    "desc":"Set the signpost to specific text",
    "icon":"Main/skin/Icons/Special/Commands/Notify.png",
    "name":"Set Signpost",
    "category":"Component"
  },
  "enable_transport_route":{
    "desc":"Enable Unit to deliver on transport route",
    "icon":"Main/skin/Icons/Common/56x56/Carry.png",
    "name":"Enable Transport Route",
    "category":"Unit"
  },
  "getfreespace":{
    "name":"Get space for item",
    "args":[["in","Item","Item to check can fit","item"],["out","Result","Number of a specific item that can fit on a unit"],["in","Unit","The unit to check (if not self)","entity",true]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Count Free Space.png",
    "desc":"Returns how many of the input item can fit in the inventory"
  },
  "make_turret_bots":{
    "name":"Make Turret Bots",
    "args":[["in","Number","Number of turret bots to maintain","num"],["exec","If Working","Where to continue if the unit started working"]],
    "key":"autobase",
    "category":"AutoBase",
    "icon":"icon_input",
    "desc":"Construct and equip turret components on available carrier bots"
  },
  "label":{
    "name":"Label",
    "args":[["in","Label","Label identifier","any"]],
    "category":"Flow",
    "icon":"Main/skin/Icons/Special/Commands/Set Register.png",
    "desc":"Labels can be jumped to from anywhere in a behavior"
  },
  "disable_transport_route":{
    "desc":"Disable Unit to deliver on transport route",
    "icon":"Main/skin/Icons/Common/56x56/Carry.png",
    "name":"Disable Transport Route",
    "category":"Unit"
  },
  "sort_storage":{
    "desc":"Sorts Storage Containers on Unit",
    "icon":"Main/skin/Icons/Common/32x32/Sort.png",
    "name":"Sort Storage",
    "category":"Unit"
  },
  "gather_information":{
    "name":"Gather Information",
    "args":[["in","Range","Range of operation","num"]],
    "key":"autobase",
    "category":"AutoBase",
    "icon":"icon_input",
    "desc":"Collect information for running the auto base controller"
  },
  "get_location":{
    "name":"Get Location",
    "args":[["in","Entity","Entity to get coordinates of","entity"],["out","Coord","Coordinate of entity"]],
    "category":"Global",
    "icon":"Main/skin/Icons/Special/Commands/Move To.png",
    "desc":"Gets location of a a seen entity"
  },
  "separate_coordinate":{
    "name":"Separate Coordinate",
    "args":[["in","Coordinate",null,"coord_num"],["out","x"],["out","y"]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Split a coordinate into x and y values"
  },
  "equip_component":{
    "name":"Equip Component",
    "args":[["exec","No Component","If you don't current hold the requested component"],["in","Component","Component to equip","comp"],["in","Slot index","Individual slot to equip component from","num",true]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Common/56x56/Home.png",
    "desc":"Equips a component if it exists"
  },
  "get_health":{
    "name":"Get Health",
    "args":[["in","Entity","Entity to check","entity"],["out","Percent","Percentage of health remaining"],["out","Current","Value of health remaining",null,true],["out","Max","Value of maximum health",null,true]],
    "category":"Math",
    "icon":"Main/skin/Icons/Common/56x56/H Value.png",
    "desc":"Gets a units health as a percentage, current and max"
  },
  "domove_async":{
    "name":"Move Unit (Async)",
    "args":[["in","Target","Unit to move to","entity"]],
    "category":"Move",
    "icon":"Main/skin/Icons/Special/Commands/Move To.png",
    "desc":"Move to another unit while continuing the program"
  },
  "remap_value":{
    "name":"Remap",
    "args":[["in","Value","Value to Remap"],["in","Input Low","Low value for input"],["in","Input High","High value for input"],["in","Target Low","Low value for target"],["in","Target high","High value for target"],["out","Result","Remapped value"]],
    "category":"Global",
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Remaps a value between two ranges"
  },
  "get_resource_item":{
    "name":"Resource Type",
    "args":[["in","Resource Node","Resource Node","entity"],["out","Resource","Resource Type"],["exec","Not Resource","Continue here if it wasn't a resource node"]],
    "category":"Global",
    "icon":"Main/skin/Icons/Special/Commands/Notify.png",
    "desc":"Gets the resource type from an resource node"
  },
  "read_radio":{
    "name":"Read Radio",
    "args":[["in","Band","The band to check for"],["out","Result","Value of the radio signal"]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Scan.png",
    "desc":"Reads the Radio signal on a specified band"
  },
  "build":{
    "name":"Place Construction",
    "args":[["in","Coordinate","Target location, or at currently location if not specified","coord_num",true],["in","Rotation","Building Rotation (0 to 3) (default 0)","num",true],["exec","Construction Failed","Where to continue if construction failed"]],
    "category":"Global",
    "icon":"Main/skin/Icons/Special/Commands/Make Order.png",
    "desc":"Places a construction site for a specific structure"
  },
  "sub":{
    "name":"Subtract",
    "args":[["in","From",null,"coord_num"],["in","Num",null,"coord_num"],["out","Result"]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Substact Numbers.png",
    "desc":"Subtracts a number or coordinate from another number or coordinate"
  },
  "lock_slots":{
    "name":"Fix Item Slots",
    "args":[["in","Item","Item type to try fixing to the slots","item_num"],["in","Slot index","Individual slot to fix","num",true]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Count Free Space.png",
    "desc":"Fix all storage slots or a specific item slot index"
  },
  "launch":{
    "desc":"Launches a satellite if equipped on an AMAC",
    "icon":"Main/skin/Icons/Special/Commands/Make Order.png",
    "name":"Launch",
    "category":"Global"
  },
  "ping":{
    "name":"Pings a Unit",
    "args":[["in","Target","Target unit","entity"]],
    "category":"Global",
    "icon":"Main/skin/Icons/Special/Commands/Notify.png",
    "desc":"Plays the Ping effect and notifies other players"
  },
  "order_transfer":{
    "name":"Order Transfer To",
    "args":[["in","Target","Target unit","entity"],["in","Item","Item and amount to transfer","item_num"]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Special/Commands/Make Order.png",
    "desc":"Transfers an Item to another Unit"
  },
  "select_nearest":{
    "name":"Select Nearest",
    "args":[["exec","A","A is nearer (or equal)"],["exec","B","B is nearer"],["in","Unit A",null,"entity"],["in","Unit B",null,"entity"],["out","Closest","Closest unit",null,true]],
    "category":"Flow",
    "icon":"Main/skin/Icons/Special/Commands/Closest Enemy.png",
    "desc":"Branches based on which unit is closer, optional branches for closer unit"
  },
  "percent_value":{
    "name":"Percent",
    "args":[["in","Value","Value to check"],["in","Max Value","Max Value to get percentage of"],["out","Number","Percent"]],
    "category":"Global",
    "icon":"Main/skin/Icons/Special/Commands/Compare Values.png",
    "desc":"Gives you the percent that value is of Max Value"
  },
  "value_type":{
    "name":"Data type switch",
    "args":[["in","Data","Data to test"],["exec","Item","Item Type"],["exec","Entity","Entity Type"],["exec","Component","Component Type"],["exec","Tech","Tech Type",null,true],["exec","Value","Information Value Type",null,true],["exec","Coord","Coordinate Value Type",null,true]],
    "category":"Flow",
    "exec_arg":[1,"No Match","Where to continue if there is no match"],
    "icon":"Main/skin/Icons/Common/56x56/Processing.png",
    "desc":"Switch based on type of value"
  },
  "check_grid_effeciency":{
    "name":"Check Grid Efficiency",
    "args":[["exec","Full","Where to continue if at full efficiency"],["in","Unit","The unit to check for (if not self)","entity",true]],
    "category":"Unit",
    "icon":"Main/skin/Icons/Common/56x56/Power.png",
    "desc":"Checks the Efficiency of the power grid the unit is on"
  },
  "moveaway_range":{
    "name":"Move Away (Range)",
    "args":[["in","Target","Unit to move away from","entity"]],
    "category":"Move",
    "icon":"Main/skin/Icons/Special/Commands/Move To.png",
    "desc":"Moves out of range of another unit"
  },
  "disconnect":{
    "desc":"Disconnects Units from Logistics Network",
    "icon":"Main/skin/Icons/Common/56x56/Carry.png",
    "name":"Disconnect",
    "category":"Unit"
  },
  "make_carrier":{
    "name":"Make Carriers",
    "args":[["in","Carriers","Type and count of carriers to make","frame_num"],["exec","If Working","Where to continue if the unit started working"]],
    "key":"autobase",
    "category":"AutoBase",
    "icon":"icon_input",
    "desc":"Construct carrier bots for delivering orders or to use for other tasks"
  },
  "wait":{
    "name":"Wait Ticks",
    "args":[["in","Time","Number of ticks to wait","num"]],
    "category":"Flow",
    "icon":"Main/skin/Icons/Special/Commands/Wait.png",
    "desc":"Pauses execution of the behavior until 1 or more ticks later"
  },
  "mul":{
    "name":"Multiply",
    "args":[["in","To",null,"coord_num"],["in","Num",null,"coord_num"],["out","Result"]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Mul Numbers.png",
    "desc":"Multiplies a number or coordinate from another number or coordinate"
  },
  "checkfreespace":{
    "name":"Check space for item",
    "args":[["exec","Can't Fit","Execution if it can't fit the item"],["in","Item","Item and amount to check can fit","item_num"]],
    "category":"Math",
    "icon":"Main/skin/Icons/Special/Commands/Count Free Space.png",
    "desc":"Checks if free space is available for an item and amount"
  },
  "move_south":{
    "name":"Move South",
    "args":[["in","Number","Number of tiles to move South","num"]],
    "category":"Move",
    "icon":"Main/skin/Icons/Special/Commands/Move To.png",
    "desc":"Moves towards a tile South of the current location at the specified distance"
  }
}
'''

In [184]:
import json

def import_desynced_ops(json_contents):
    raw_import = json.loads(json_contents)
    instructions = {to_function_name(v['name']):{**v, 'op':k} for k,v in raw_import.items() if 'name' in v.keys()}
    return instructions

    
def to_function_name(string):
    # Desynced block names are typically all capitalized.  Standardize on this.
    # Obsolete instructions are marked by being surrounded by asterixes.  Replace with _ so it is a valid python function name
    result =''.join([word.capitalize() for word in string.replace('*','_').replace('(','').replace(')','').split()])
    return result


ds_ops = import_desynced_ops(contents_of_instructionsjson)

## 2. Generate an Abstract Syntax Tree (AST)
Use part of the existing Python tool chain to partially compile Python into an Abstract Syntax Tree (AST). The AST represents the structure of the Python code, allowing us to analyze and manipulate it programmatically.  

I wrapped astprettyprint because my transforms mangle offsets.  
Feel free to try AST parsing any python code you'd like - either define a function and reference it (example 1) or put it in a string and pass it directly (example 2).  
It is worth spending some time getting comfortable at this step.

In [259]:
import ast, inspect

# astpretty is much better looking, install it!

try:
    import astpretty

    # simple wrapper of the astprettyprint library
    def astprint(tree):
        astpretty.pprint(tree, show_offsets=False)
        
except ModuleNotFoundError:
    import pprint
    def ast_to_dict(node):
        if isinstance(node, ast.AST):
            node_dict = {'type': type(node).__name__}
            for field, value in ast.iter_fields(node):
                node_dict[field] = ast_to_dict(value)
            return node_dict
        elif isinstance(node, list):
            return [ast_to_dict(item) for item in node]
        else:
            return node

    def astprint(tree):
        pprint.pprint(ast_to_dict(tree))



In [260]:
def DesyncedCode(P2):
    for A in LoopSignalMatch(P2):
        P3 = SelectNearest(A, P3)
    Move(GetLocation(P3))
    return P1

source_code = inspect.getsource(DesyncedCode)
tree = ast.parse(source_code, type_comments=False)
print("AST Tree from inspected source:")
astprint(tree)

source_code_by_string="""
A=B+C+D
"""
tree = ast.parse(tree, type_comments=False)
print("\n\nAST Tree from string source:")
astprint(tree)


AST Tree from inspected source:
Module(
    body=[
        FunctionDef(
            name='DesyncedCode',
            args=arguments(
                posonlyargs=[],
                args=[arg(arg='P2', annotation=None, type_comment=None)],
                vararg=None,
                kwonlyargs=[],
                kw_defaults=[],
                kwarg=None,
                defaults=[],
            ),
            body=[
                For(
                    target=Name(id='A', ctx=Store()),
                    iter=Call(
                        func=Name(id='LoopSignalMatch', ctx=Load()),
                        args=[Name(id='P2', ctx=Load())],
                        keywords=[],
                    ),
                    body=[
                        Assign(
                            targets=[Name(id='P3', ctx=Store())],
                            value=Call(
                                func=Name(id='SelectNearest', ctx=Load()),
                                args=[
      

## 3. Replace +-*/ with Function calls Add, Subtract, etc.

Python supports so many different ways to express things, this step narrows that down a bit.  
BinOps like A+B get replaced with Calls like Add(A,B).  
This is done early so that all function calls in the AST use the same syntax, which makes the rest of the code easier.

Try experimenting with order of operations and see if the resulting tree makes sense.

In [261]:
def replace_binops_with_functions(tree):
    class BinOpReplacementVisitor(ast.NodeTransformer):
        # Replace binary operations with function calls
        binop_map = {ast.Add: 'Add',
                     ast.Sub: 'Subtract',
                     ast.Mult:'Multiply',
                     ast.Div: 'Divide',
                     ast.Mod: 'Modulo',
                        }
        def visit_BinOp(self, node):
            if isinstance(node.op, ast.operator) and node.op.__class__ in self.binop_map:
                #self.generic_visit(node)
                new_node = ast.Call(
                    func=ast.Name(id=self.binop_map[type(node.op)], ctx=ast.Load()),
                    args=[node.left, node.right],
                    keywords=[],
                    starargs=None,
                    kwargs=None
                )
                new_node=self.visit(new_node)
                return new_node
            else:
                return node

        def visit_AugAssign(self, node):
            if isinstance(node.op, ast.operator) and node.op.__class__ in self.binop_map:
                new_node = ast.Assign(
                    targets = [node.target],
                    value = ast.Call(
                        func=ast.Name(id=self.binop_map[type(node.op)], ctx=ast.Load()),
                        args=[node.target, node.value],
                        keywords=[],
                        starargs=None,
                        kwargs=None
                    )
                )
                
                new_node=self.visit(new_node)
                return new_node
            else:
                return node
                
    # Perform transformations until no changes are made
    # I hate this... did I screw up recursion somewhere?

    visitor=BinOpReplacementVisitor()
    return visitor.visit(tree)
    
    while True:
        old_tree_str = ast.dump(tree)
        # Create a visitor
        visitor = BinOpReplacementVisitor()
        # Visit the tree and get the transformed tree
        new_tree = visitor.visit(tree)
        new_tree_str = ast.dump(new_tree)
        # Check if any changes were made
        if old_tree_str == new_tree_str:
            # No changes were made, exit the loop
            break
        # Update the tree for the next pass
        tree = new_tree
    return tree


In [253]:
source_code='''
A+=B
A=A+B
A=B*C+D/E-F
A=Beta(C,D+1+3)
'''

tree = ast.parse(source_code, type_comments=False)
astprint(tree)
print('\n\n')
tree = replace_binops_with_functions(tree)
astprint(tree)

Module(
    body=[
        AugAssign(
            target=Name(id='A', ctx=Store()),
            op=Add(),
            value=Name(id='B', ctx=Load()),
        ),
        Assign(
            targets=[Name(id='A', ctx=Store())],
            value=BinOp(
                left=Name(id='A', ctx=Load()),
                op=Add(),
                right=Name(id='B', ctx=Load()),
            ),
            type_comment=None,
        ),
        Assign(
            targets=[Name(id='A', ctx=Store())],
            value=BinOp(
                left=BinOp(
                    left=BinOp(
                        left=Name(id='B', ctx=Load()),
                        op=Mult(),
                        right=Name(id='C', ctx=Load()),
                    ),
                    op=Add(),
                    right=BinOp(
                        left=Name(id='D', ctx=Load()),
                        op=Div(),
                        right=Name(id='E', ctx=Load()),
                    ),
                ),
  

# 4. Flatten Nested Calls 
Desynced's execution enviornment isn't as powerful as Python's and does not support nested function calls. This step flattens code using temporary variables:  
```python  
A = Distance()
   -->
temp1 = GetClosestEntity("Enemy")
A     = Distance(temp1)
```
# 5.  Translate all function calls to a generic "Desynced Call" AST type.  
Defining a custom AST node type allows us to decorate it with additional information that will later be useful for compilation.  
Specifically, each DS Function containsan *opcode*, *inputs*, *outputs*, and *execution flows*.  
Python's AST includes the inputs as part of the call node, but outputs are held in an Assign node and execution flow is handled fundamentally different.  


In [364]:
class DS_Call(ast.Assign):
    _fields = ('targets', 'args', 'op', 'frame', 'next')

    def __init__(self, targets, args, op):
        self.targets = targets
        self.args = args
        self.next = -1
        self.frame = -1
        self.op = op

def transform_nested_calls(tree):
    class FlatteningTransformer(ast.NodeTransformer):  
        def __init__(self):
            super().__init__()
            self.temp_count = 1
            
        def visit_Call(self, node, target=None):
            if target:
                node.targets=[target]
            
            nodelist = [DS_Call( targets=[target], args=node.args, op=node.func.id)]
            for i, arg in enumerate(node.args):
                
                if isinstance(arg, ast.Call):
                    temp_var = f'Temp_{self.temp_count}'
                    self.temp_count += 1                    
                    nodelist.insert(0, self.visit_Call(arg, ast.Name(id=temp_var, ctx=ast.Store())))
                    node.args[i] = ast.Name(id=temp_var, ctx=ast.Load())
                    
            # Return a list containing the original call node and the new call node
            def flatten(lst):
                return [item for sublist in lst for item in (flatten(sublist) if isinstance(sublist, list) else [sublist])]
            nodelist = flatten(nodelist)
            for node in nodelist:
                self.visit(node)
            return nodelist

        def visit_Assign(self, node):

            if isinstance(node.value, (ast.Name, ast.Constant, ast.Tuple)):
                
                # Special case for a bare Assignment - this is a Copy function, which is 'set_reg' internally
                new_node = DS_Call(targets = node.targets, args = [node.value], op = 'Copy')
                new_node = self.visit(new_node)
                return new_node
            nodelist = self.visit_Call(node.value)
            nodelist[-1].targets = node.targets
            for n in nodelist:
                self.visit(n)
            return nodelist

        def visit_Expr(self, node):
            nodelist = self.visit_Call(node.value)
            for n in nodelist:
                self.visit(n)
            return nodelist

        def visit_Tuple(self, node):
            if isinstance(node.elts[0], ast.Name):
                return [e for e in node.elts]
            if isinstance(node.elts[0], ast.Constant):
                return ast.Constant( value=[e.value for e in node.elts])
            return node

    return FlatteningTransformer().visit(tree)

In [338]:
source_code='''
A=("Ore", 3)

A=B
A=A+1
A = B+C+D+E
if CompareNumber(GetDistance(P1), 3):
    P2 = A+1
A=B
'''

tree = ast.parse(source_code, type_comments=False)
astprint(tree)
print('\n\n')
tree = replace_binops_with_functions(tree)
tree = transform_nested_calls(tree)
astprint(tree)

Module(
    body=[
        Assign(
            targets=[Name(id='A', ctx=Store())],
            value=Tuple(
                elts=[
                    Constant(value='Ore', kind=None),
                    Constant(value=3, kind=None),
                ],
                ctx=Load(),
            ),
            type_comment=None,
        ),
        Assign(
            targets=[Name(id='A', ctx=Store())],
            value=Name(id='B', ctx=Load()),
            type_comment=None,
        ),
        Assign(
            targets=[Name(id='A', ctx=Store())],
            value=BinOp(
                left=Name(id='A', ctx=Load()),
                op=Add(),
                right=Constant(value=1, kind=None),
            ),
            type_comment=None,
        ),
        Assign(
            targets=[Name(id='A', ctx=Store())],
            value=BinOp(
                left=BinOp(
                    left=BinOp(
                        left=Name(id='B', ctx=Load()),
                        op=Add()

# 6. Labeling

Any illegal variable names throw a compilation error at this step.  
Temporary variables are renamed.  
All DS_Calls are assigned a frame number, which is the desynced equivalent of a line number.

This is accomplished by first enumerating all of the known variables in the code (VariableFinder).  Then find the lowest unused local variable, and remap temporary variables starting there.  I don't do any form of variable lifespan checking - This is inefficient but usable.  
Then we use another NodeTransformer to visit every DS_Call and give it a frame id number.  Fortunately the NodeTransformer walks the tree in a reasonable manner, which makes it easier to visually inspect later.  This could work with pseudorandom assignment, but this way is cleaner.


In [302]:
def label_frames(tree):
    def create_variable_remap(input_list):
        highest_letter = 'A'
        highest_temp_number = 0
    
        for item in input_list:
            if item.startswith('Temp_'):
                temp_number = int(item.split('_')[1])
                highest_temp_number = max(highest_temp_number, temp_number)
            elif item.isalpha():
                highest_letter = max(highest_letter, item)

        mapping = {i:i for i in input_list}
        for i in range(1, highest_temp_number + 1):
            temp_key = f'Temp_{i}'
            temp_value = chr(ord(highest_letter) + i)
            mapping[temp_key] = temp_value
    
        return mapping

    class VariableFinder(ast.NodeVisitor):
        def __init__(self):
            self.variables = {}

        def visit_Name(self, node):
            self.variables[node.id]=None

    class VariableLabeler(ast.NodeTransformer):
        def __init__(self, mapper):
            super().__init__()
            self.mapper = mapper
        
        def visit_Name(self, node):
            if node.id in self.mapper.keys():
                node.id = self.mapper[node.id]
            return node
            
    class FrameLabeler(ast.NodeTransformer):  
        def __init__(self):
            super().__init__()
            self.frame_count = 0
            
        def visit_DS_Call(self, node):
            node.frame = self.frame_count
            self.frame_count+=1
            return node

            
    finder = VariableFinder()
    finder.visit(tree)
    vars = finder.variables.keys()
    remap = create_variable_remap(vars)
    varlabeler = VariableLabeler(remap)
    tree = varlabeler.visit(tree)
    framelabeler = FrameLabeler()
    tree = framelabeler.visit(tree)
    return tree

In [303]:
source_code='''
A=B
A=A+1
A = B+C+D+E
if CompareNumber(GetDistance(P1), 3):
    P2 = A+1
A=B
'''


tree = ast.parse(source_code, type_comments=False)

tree = replace_binops_with_functions(tree)
tree = transform_nested_calls(tree)
astprint(tree)
print('\n\n')
tree = label_frames(tree)
astprint(tree)

Module(
    body=[
        DS_Call(
            targets=[Name(id='A', ctx=Store())],
            args=Name(id='B', ctx=Load()),
            op='set_reg',
            frame=-1,
            next=-1,
        ),
        DS_Call(
            targets=[Name(id='A', ctx=Store())],
            args=[
                Name(id='A', ctx=Load()),
                Constant(value=1, kind=None),
            ],
            op='Add',
            frame=-1,
            next=-1,
        ),
        DS_Call(
            targets=[Name(id='Temp_2', ctx=Store())],
            args=[
                Name(id='B', ctx=Load()),
                Name(id='C', ctx=Load()),
            ],
            op='Add',
            frame=-1,
            next=-1,
        ),
        DS_Call(
            targets=[Name(id='Temp_1', ctx=Store())],
            args=[
                Name(id='Temp_2', ctx=Load()),
                Name(id='D', ctx=Load()),
            ],
            op='Add',
            frame=-1,
            next=-1,
    

# 7. Flow Control

Flow control dictates the order in which these ops are called.  The linear sections are easy - just link each frame to the next one.
The first and last frame in a sequence are special.  The first frame is where other sequences link into a sequence, the last frame is where it links out.  
The helper function makes it easier to identify these special frames, and then each of the flow control elements (IF, While, For) have to be special cased.

I don't use a Transformer class here so that I have better access to the parents of a node that is being visited.

In [358]:
def find_first_last_DS_Call(list):
    class DSFinder(ast.NodeVisitor):
        def __init__(self):
            self.last=None
            self.first= None

        def visit_DS_Call(self, node):

            self.last = node
            if self.first is None:
                self.first = node

    finder = DSFinder()
    for tree in list:
        finder.visit(tree)
    return finder.first, finder.last

def flow_list(nodelist, exit=-1):
    #myexit = find_first_last_DS_Call(nodelist)
    for item, next_item in zip(nodelist, nodelist[1:] + [None]):
        lexit = exit if next_item is None else find_first_last_DS_Call([next_item])[0].frame
        if isinstance(item, DS_Call):
            item.next = {'next': lexit}
        if isinstance(item, ast.While):
            flow_While(item, lexit)
        if isinstance(item, ast.If):
            flow_If(item, lexit)
        if isinstance(item, ast.For):
            flow_For(item, lexit)

def flow_While(node, exit=-1):
    flow_list(node.test)
    flow_list(node.body)
    body_first, body_last = find_first_last_DS_Call(node.body)
    test_first, test_last = find_first_last_DS_Call(node.test)

    test_last.next = {'next': body_first.frame, 'exit': exit}
    body_last.next = {'next': test_first.frame,}

def flow_If(node, exit=-1):
    flow_list(node.test)
    flow_list(node.body)
    flow_list(node.orelse)
    body_first, body_last = find_first_last_DS_Call(node.body)
    test_first, test_last = find_first_last_DS_Call(node.test)
    orelse_first, orelse_last = find_first_last_DS_Call(node.orelse)

    test_last.next = {'next': exit if body_first is None else body_first.frame,
                      'else': exit if orelse_first is None else orelse_first.frame}
    if body_last:
        body_last.next = {'next': exit}
    if orelse_last:
        orelse_last.next = {'next':exit}


def flow_For(node, exit=-1):
    target = node.target
    node.target = None
    #print('For:')
    #astprint(node)
    flow_list(node.body)
    flow_list(node.iter)
    body_first, body_last = find_first_last_DS_Call(node.body)
    iter_first, iter_last = find_first_last_DS_Call(node.iter)

    iter_last.next = {'next': body_first.frame, 
                      'exit': exit}
    body_last.next= {'next': iter_first.frame}
    if isinstance(target, list):
        iter_last.targets = target
    else:
        iter_last.targets = [target]

def flow_control(tree):
    return flow_list(tree.body)
    

In [357]:
source_code='''

while CheckBattery(GetSelf()):
    B=B+1
    A=1+2+3
for C in LoopSignalMatch(P3):
   P4=("Ore", 17)
'''

source_code='''
for C,D in LoopSignalMatch(P3):
   P4=C'''

tree = ast.parse(source_code, type_comments=False)

tree = replace_binops_with_functions(tree)
tree = transform_nested_calls(tree)
tree = label_frames(tree)
#astprint(tree)
#print('\n\n')
flow_control(tree)
astprint(tree)

For:
For(
    target=None,
    iter=[
        DS_Call(
            targets=[
                None,
            ],
            args=[Name(id='P3', ctx=Load())],
            op='LoopSignalMatch',
            frame=0,
            next=-1,
        ),
    ],
    body=[
        DS_Call(
            targets=[Name(id='P4', ctx=Store())],
            args=Name(id='C', ctx=Load()),
            op='set_reg',
            frame=1,
            next=-1,
        ),
    ],
    orelse=[],
    type_comment=None,
)
Module(
    body=[
        For(
            target=None,
            iter=[
                DS_Call(
                    targets=[
                        Name(id='C', ctx=Store()),
                        Name(id='D', ctx=Store()),
                    ],
                    args=[Name(id='P3', ctx=Load())],
                    op='LoopSignalMatch',
                    frame=0,
                    next={'next': 1, 'exit': -1},
                ),
            ],
            body=[
               

# 8. Generate as JSON

The modified AST tree now has all the information necessary to emit actual op codes.  Desynced uses JSON as an intermediate langauge, but it was not designed for readability.  All descriptive field names are stripped and replaced with numerical indices.  


In [498]:
def walk(tree):
    def translate_register_or_value(val):
        if isinstance(val, ast.Name):
            return val.id
        if isinstance(val, ast.Constant):
            return {"num": val.value}
    
    class DSCall_to_JSON(ast.NodeVisitor):
        def __init__(self, debug=False):
            self.frames={}
            self.debug=debug

        def visit_DS_Call(self, node):
            op = ds_ops[node.op]
            if self.debug:
                print('\n\n')
                print(op)
                astprint(node)
            res={}
            res['op'] = op['op']
            if node.next['next'] != node.frame+1:
                res['next'] = node.next['next']
            target_ix=0
            arg_ix=0
            exec_ix=1
            if 'args' in op.keys():
                if self.debug:
                    print(op['args'])
                for i, arg in enumerate(op['args']):
                    if arg[0] == 'out':
                        res[str(i)] = translate_register_or_value(node.targets[target_ix])
                        target_ix +=1
                    if arg[0] == 'in':
                        res[str(i)] = translate_register_or_value(node.args[arg_ix])
                        arg_ix +=1
                    if arg[0] == 'exec':

                        res[str(i)]=list(node.next.values())[exec_ix]+1
                        exec_ix+=1
                        
            
            self.frames[str(node.frame)] = res


    walker = DSCall_to_JSON()
    walker.visit(tree)

    return walker.frames

In [499]:
source_code='''
for A, B in LoopSignalMatch(P3):
    P4=C
    
while CheckBattery(GetSelf()):
    B=B+1
    A=1+2+3'''
source_code='''
if IsFixed(P1):
    A=A+1
else:
    B=B+1
'''

tree = ast.parse(source_code, type_comments=False)

tree = replace_binops_with_functions(tree)
tree = transform_nested_calls(tree)
tree = label_frames(tree)


flow_control(tree)
astprint(tree)
frames = walk(tree)





Module(
    body=[
        If(
            test=[
                DS_Call(
                    targets=[
                        None,
                    ],
                    args=[Name(id='P1', ctx=Load())],
                    op='IsFixed',
                    frame=0,
                    next={'next': 1, 'else': 2},
                ),
            ],
            body=[
                DS_Call(
                    targets=[Name(id='A', ctx=Store())],
                    args=[
                        Name(id='A', ctx=Load()),
                        Constant(value=1, kind=None),
                    ],
                    op='Add',
                    frame=1,
                    next={'next': -1},
                ),
            ],
            orelse=[
                DS_Call(
                    targets=[Name(id='B', ctx=Store())],
                    args=[
                        Name(id='B', ctx=Load()),
                        Constant(value=1, kind=None),
                    ]

In [403]:
jsonstring = json.dumps(frames)
print(jsonstring)


{"0": {"op": "is_fixed", "0": "P1", "1": 3}, "1": {"op": "add", "next": -1, "0": "A", "1": {"num": 1}, "2": "A"}, "2": {"op": "add", "next": -1, "0": "B", "1": {"num": 1}, "2": "B"}}


# 9. Base62 Encode  
Desynced has it's own variant of the Base62 encoding format that is uses to make these json files easily copy pasted on the internet.  As a last step we shove the JSON into this format so it can be copied into the game.


In [507]:
%%javascript

var jsonstring={{jsonstring}}

/*
   Copyright (C) 2023-2024 Stage Games Inc.

   Permission is hereby granted, free of charge, to any person obtaining a copy of this
   software and associated documentation files (the "Software"), to deal in the Software
   without restriction, including without limitation the rights to use, copy, modify, merge,
   publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons
   to whom the Software is furnished to do so, subject to the following conditions:

   The above copyright notice and this permission notice shall be included in all
   copies or substantial portions of the Software.

   THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED,
   INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR
   PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE
   LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT,
   TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE
   USE OR OTHER DEALINGS IN THE SOFTWARE.
*/



function ObjectToDesyncedString(obj, type)
{
	// Serialize object into growing buffer
	var mem = new WebAssembly.Memory({initial: 1, maximum: 4096 }); // 64kb pages
	var bytes = new Uint8Array(mem.buffer), view = new DataView(mem.buffer), pos = 0, end = 0, compressed_length = 0;
	const utf8 = new (typeof process === 'object' ? require("util").TextEncoder : TextEncoder)(); // default 'utf-8' or 'utf8'

	const
		MP_FixZero   = 0x00, MP_FixMap    = 0x80, MP_FixArray  = 0x90, MP_FixStr    = 0xa0,
		MP_Nil       = 0xc0, MP_False     = 0xc2, MP_True      = 0xc3,
		MP_Float32   = 0xca, MP_Float64   = 0xcb,
		MP_Uint8     = 0xcc, MP_Uint16    = 0xcd, MP_Uint32    = 0xce, MP_Uint64    = 0xcf,
		MP_Int8      = 0xd0, MP_Int16     = 0xd1, MP_Int32     = 0xd2, MP_Int64     = 0xd3,
		MP_Str8      = 0xd9, MP_Str16     = 0xda, MP_Str32     = 0xdb,
		MP_Array16   = 0xdc, MP_Array32   = 0xdd,
		MP_Map16     = 0xde, MP_Map32     = 0xdf;

	function Serialize(v, is_table_key)
	{
		function Grow(n)
		{
			const up = (end = (pos = end) + n) - bytes.length;
			if (up > 0) { mem.grow((up+0xFFFF)>>16); bytes = new Uint8Array(mem.buffer); view = new DataView(mem.buffer); }
			return view;
		}
		function Push(v) { Grow(1).setUint8(pos, v); }
		function PushIntPacked(v) { do { var b = v & 127; v >>= 7; Push((b << 1) | (v ? 1 : 0)); } while (v); }

        console.log(v, typeof(v))
		switch (v == null ? 'undefined' : typeof(v))
		{
			case 'undefined':
				if (is_table_key) throw new Error("Unable to serialize table key of type 'null/undefined'");
				Push(MP_Nil); 
				break;
			case 'boolean':
				if (is_table_key) throw new Error("Unable to serialize table key of type 'boolean'");
				Push(v ? MP_True : MP_False);
				break;
			case 'number':
				if (!Number.isInteger(v))  { Push(MP_Float64); Grow(8).setFloat64(pos, v, true); }
				else if (v >  0xffffffff)  { Push(MP_Uint64); Grow(8).setUint64(pos, v, true); }
				else if (v >  0xffff)      { Push(MP_Uint32); Grow(4).setUint32(pos, v, true); }
				else if (v >  0xff)        { Push(MP_Uint16); Grow(2).setUint16(pos, v, true); }
				else if (v >  0x7F)        { Push(MP_Uint8);  Grow(1).setUint8(pos, v); }
				else if (v >= 0)           { Push(v); }
				else if (v >= -32)         { Push(v + 256); }
				else if (v >= -128)        { Push(MP_Int8); Grow(1).setInt8(pos, v); }
				else if (v >= -32768)      { Push(MP_Int16); Grow(2).setInt16(pos, v, true); }
				else if (v >= -2147483648) { Push(MP_Int32); Grow(4).setInt32(pos, v, true); }
				else                       { Push(MP_Uint64); Grow(8).setUint64(pos, v, true); }
				break;
			case 'string':
				const strsz = v.length;
				if      (strsz <    32) { Push(MP_FixStr | strsz); }
				else if (strsz <   256) { Push(MP_Str8); Grow(1).setUint8(pos, strsz); }
				else if (strsz < 65536) { Push(MP_Str16); Grow(2).setUint16(pos, strsz, true); }
				else                    { Push(MP_Str32); Grow(4).setUint32(pos, strsz, true); }
				const encoded = utf8.encode(v);
				Grow(encoded.length);
				bytes.set(encoded, pos);
				break;
			case 'object':
				if (is_table_key) throw new Error("Unable to serialize table key of type 'table'");
				for (var size_node = 0, size_array = 0, array_keys = 0;; size_array++)
				{
					if (v.hasOwnProperty(size_array)) array_keys++;
					else if (!v.hasOwnProperty(size_array+1)) break; // allow 1 gap
				}
				var keys = Object.keys(v), key_count = keys.length, map_keys = key_count - array_keys;
				var sz = (map_keys ? (((key_count - array_keys - 1).toString(2).length << 1) | (size_array ? 1 : 0)) : size_array);
				if      (sz <    16) { Push((map_keys ? MP_FixMap : MP_FixArray) | sz); }
				else if (sz < 65536) { Push((map_keys ? MP_Map16 : MP_Array16)); Grow(2).setUint16(pos, sz, true); }
				else                 { Push((map_keys ? MP_Map32 : MP_Array32)); Grow(4).setUint32(pos, sz, true); }

				if (map_keys)
				{
					size_node = (1 << (sz >> 1)); // always the next power of 2
					if (size_array)
					{
						PushIntPacked(size_array); // store length of array part
						keys = keys.filter((k)=> !(k >= 0 && k < size_array)); // filter array-number keys
					}
					Push(0); // used for Lua table memory layout, ignored by the game for incoming encoded strings
				}

				for (var i = 0, total = size_array + size_node, last = size_array + map_keys, vacancy_bits = 0; i != total;)
				{
					var bit = (i & 7), vacant = (i >= last || (i < size_array && v[i] === undefined));
					vacancy_bits = ((bit ? vacancy_bits : 0) | (vacant << bit));
					if (++i != total && bit != 7) continue;

					// Write out bit+1 elements
					Push(vacancy_bits);
					for (var j = i - 1 - bit; j != i; j++)
					{
						if (j >= last || (j < size_array && v[j] === undefined)) continue; // vacant

						// Write out array/object value
						Serialize(j < size_array ? v[j] : v[keys[j - size_array]]);
						if (j < size_array) continue;

						// Write out object key
						var key = keys[j - size_array];
						Serialize(/\D/.test(key) ? key : ((key|0) + 1), true); // convert numerical keys to 1-based indexing
						Push(0); // used for Lua table memory layout, ignored by the game for incoming encoded strings
					}
				}
				break;

			default: throw new Error('cannot parse unsupported type ' + typeof(v));
		}
	}


	Serialize(obj);
	// Compress the output, but only use the compressed format if it actually saves space
	if (1)
	{
		/** @license zlib.js 2012 - imaya [ https://github.com/imaya/zlib.js ] The MIT License */
		var Zlib = {}, comp; (function() {'use strict';var n=void 0,w=!0,aa=this;function ba(f,d){var c=f.split("."),e=aa;!(c[0]in e)&&e.execScript&&e.execScript("var "+c[0]);for(var b;c.length&&(b=c.shift());)!c.length&&d!==n?e[b]=d:e=e[b]?e[b]:e[b]={}};var C="undefined"!==typeof Uint8Array&&"undefined"!==typeof Uint16Array&&"undefined"!==typeof Uint32Array&&"undefined"!==typeof DataView;function K(f,d){this.index="number"===typeof d?d:0;this.e=0;this.buffer=f instanceof(C?Uint8Array:Array)?f:new (C?Uint8Array:Array)(32768);if(2*this.buffer.length<=this.index)throw Error("invalid index");this.buffer.length<=this.index&&ca(this)}function ca(f){var d=f.buffer,c,e=d.length,b=new (C?Uint8Array:Array)(e<<1);if(C)b.set(d);else for(c=0;c<e;++c)b[c]=d[c];return f.buffer=b}K.prototype.b=function(f,d,c){var e=this.buffer,b=this.index,a=this.e,g=e[b],m;c&&1<d&&(f=8<d?(L[f&255]<<24|L[f>>>8&255]<<16|L[f>>>16&255]<<8|L[f>>>24&255])>>32-d:L[f]>>8-d);if(8>d+a)g=g<<d|f,a+=d;else for(m=0;m<d;++m)g=g<<1|f>>d-m-1&1,8===++a&&(a=0,e[b++]=L[g],g=0,b===e.length&&(e=ca(this)));e[b]=g;this.buffer=e;this.e=a;this.index=b};K.prototype.finish=function(){var f=this.buffer,d=this.index,c;0<this.e&&(f[d]<<=8-this.e,f[d]=L[f[d]],d++);C?c=f.subarray(0,d):(f.length=d,c=f);return c};var da=new (C?Uint8Array:Array)(256),M;for(M=0;256>M;++M){for(var N=M,S=N,ea=7,N=N>>>1;N;N>>>=1)S<<=1,S|=N&1,--ea;da[M]=(S<<ea&255)>>>0}var L=da;function ia(f){this.buffer=new (C?Uint16Array:Array)(2*f);this.length=0}ia.prototype.getParent=function(f){return 2*((f-2)/4|0)};ia.prototype.push=function(f,d){var c,e,b=this.buffer,a;c=this.length;b[this.length++]=d;for(b[this.length++]=f;0<c;)if(e=this.getParent(c),b[c]>b[e])a=b[c],b[c]=b[e],b[e]=a,a=b[c+1],b[c+1]=b[e+1],b[e+1]=a,c=e;else break;return this.length};ia.prototype.pop=function(){var f,d,c=this.buffer,e,b,a;d=c[0];f=c[1];this.length-=2;c[0]=c[this.length];c[1]=c[this.length+1];for(a=0;;){b=2*a+2;if(b>=this.length)break;b+2<this.length&&c[b+2]>c[b]&&(b+=2);if(c[b]>c[a])e=c[a],c[a]=c[b],c[b]=e,e=c[a+1],c[a+1]=c[b+1],c[b+1]=e;else break;a=b}return{index:f,value:d,length:this.length}};function ka(f,d){this.d=la;this.i=0;this.input=C&&f instanceof Array?new Uint8Array(f):f;this.c=0;d&&(d.lazy&&(this.i=d.lazy),"number"===typeof d.compressionType&&(this.d=d.compressionType),d.outputBuffer&&(this.a=C&&d.outputBuffer instanceof Array?new Uint8Array(d.outputBuffer):d.outputBuffer),"number"===typeof d.outputIndex&&(this.c=d.outputIndex));this.a||(this.a=new (C?Uint8Array:Array)(32768))}var la=2,na={NONE:0,h:1,g:la,n:3},T=[],U;for(U=0;288>U;U++)switch(w){case 143>=U:T.push([U+48,8]);break;case 255>=U:T.push([U-144+400,9]);break;case 279>=U:T.push([U-256+0,7]);break;case 287>=U:T.push([U-280+192,8]);break;default:throw"invalid literal: "+U;}ka.prototype.f=function(){var f,d,c,e,b=this.input;switch(this.d){case 0:c=0;for(e=b.length;c<e;){d=C?b.subarray(c,c+65535):b.slice(c,c+65535);c+=d.length;var a=d,g=c===e,m=n,k=n,p=n,t=n,u=n,l=this.a,h=this.c;if(C){for(l=new Uint8Array(this.a.buffer);l.length<=h+a.length+5;)l=new Uint8Array(l.length<<1);l.set(this.a)}m=g?1:0;l[h++]=m|0;k=a.length;p=~k+65536&65535;l[h++]=k&255;l[h++]=k>>>8&255;l[h++]=p&255;l[h++]=p>>>8&255;if(C)l.set(a,h),h+=a.length,l=l.subarray(0,h);else{t=0;for(u=a.length;t<u;++t)l[h++]=a[t];l.length=h}this.c=h;this.a=l}break;case 1:var q=new K(C?new Uint8Array(this.a.buffer):this.a,this.c);q.b(1,1,w);q.b(1,2,w);var s=oa(this,b),x,fa,z;x=0;for(fa=s.length;x<fa;x++)if(z=s[x],K.prototype.b.apply(q,T[z]),256<z)q.b(s[++x],s[++x],w),q.b(s[++x],5),q.b(s[++x],s[++x],w);else if(256===z)break;this.a=q.finish();this.c=this.a.length;break;case la:var B=new K(C?new Uint8Array(this.a.buffer):this.a,this.c),ta,J,O,P,Q,La=[16,17,18,0,8,7,9,6,10,5,11,4,12,3,13,2,14,1,15],X,ua,Y,va,ga,ja=Array(19),wa,R,ha,y,xa;ta=la;B.b(1,1,w);B.b(ta,2,w);J=oa(this,b);X=pa(this.m,15);ua=qa(X);Y=pa(this.l,7);va=qa(Y);for(O=286;257<O&&0===X[O-1];O--);for(P=30;1<P&&0===Y[P-1];P--);var ya=O,za=P,F=new (C?Uint32Array:Array)(ya+za),r,G,v,Z,E=new (C?Uint32Array:Array)(316),D,A,H=new (C?Uint8Array:Array)(19);for(r=G=0;r<ya;r++)F[G++]=X[r];for(r=0;r<za;r++)F[G++]=Y[r];if(!C){r=0;for(Z=H.length;r<Z;++r)H[r]=0}r=D=0;for(Z=F.length;r<Z;r+=G){for(G=1;r+G<Z&&F[r+G]===F[r];++G);v=G;if(0===F[r])if(3>v)for(;0<v--;)E[D++]=0,H[0]++;else for(;0<v;)A=138>v?v:138,A>v-3&&A<v&&(A=v-3),10>=A?(E[D++]=17,E[D++]=A-3,H[17]++):(E[D++]=18,E[D++]=A-11,H[18]++),v-=A;else if(E[D++]=F[r],H[F[r]]++,v--,3>v)for(;0<v--;)E[D++]=F[r],H[F[r]]++;else for(;0<v;)A=6>v?v:6,A>v-3&&A<v&&(A=v-3),E[D++]=16,E[D++]=A-3,H[16]++,v-=A}f=C?E.subarray(0,D):E.slice(0,D);ga=pa(H,7);for(y=0;19>y;y++)ja[y]=ga[La[y]];for(Q=19;4<Q&&0===ja[Q-1];Q--);wa=qa(ga);B.b(O-257,5,w);B.b(P-1,5,w);B.b(Q-4,4,w);for(y=0;y<Q;y++)B.b(ja[y],3,w);y=0;for(xa=f.length;y<xa;y++)if(R=f[y],B.b(wa[R],ga[R],w),16<=R){y++;switch(R){case 16:ha=2;break;case 17:ha=3;break;case 18:ha=7;break;default:throw"invalid code: "+R;}B.b(f[y],ha,w)}var Aa=[ua,X],Ba=[va,Y],I,Ca,$,ma,Da,Ea,Fa,Ga;Da=Aa[0];Ea=Aa[1];Fa=Ba[0];Ga=Ba[1];I=0;for(Ca=J.length;I<Ca;++I)if($=J[I],B.b(Da[$],Ea[$],w),256<$)B.b(J[++I],J[++I],w),ma=J[++I],B.b(Fa[ma],Ga[ma],w),B.b(J[++I],J[++I],w);else if(256===$)break;this.a=B.finish();this.c=this.a.length;break;default:throw"invalid compression type";}return this.a};function ra(f,d){this.length=f;this.k=d}var sa=function(){function f(b){switch(w){case 3===b:return[257,b-3,0];case 4===b:return[258,b-4,0];case 5===b:return[259,b-5,0];case 6===b:return[260,b-6,0];case 7===b:return[261,b-7,0];case 8===b:return[262,b-8,0];case 9===b:return[263,b-9,0];case 10===b:return[264,b-10,0];case 12>=b:return[265,b-11,1];case 14>=b:return[266,b-13,1];case 16>=b:return[267,b-15,1];case 18>=b:return[268,b-17,1];case 22>=b:return[269,b-19,2];case 26>=b:return[270,b-23,2];case 30>=b:return[271,b-27,2];case 34>=b:return[272,b-31,2];case 42>=b:return[273,b-35,3];case 50>=b:return[274,b-43,3];case 58>=b:return[275,b-51,3];case 66>=b:return[276,b-59,3];case 82>=b:return[277,b-67,4];case 98>=b:return[278,b-83,4];case 114>=b:return[279,b-99,4];case 130>=b:return[280,b-115,4];case 162>=b:return[281,b-131,5];case 194>=b:return[282,b-163,5];case 226>=b:return[283,b-195,5];case 257>=b:return[284,b-227,5];case 258===b:return[285,b-258,0];default:throw"invalid length: "+b;}}var d=[],c,e;for(c=3;258>=c;c++)e=f(c),d[c]=e[2]<<24|e[1]<<16|e[0];return d}(),Ha=C?new Uint32Array(sa):sa;function oa(f,d){function c(b,c){var a=b.k,d=[],e=0,f;f=Ha[b.length];d[e++]=f&65535;d[e++]=f>>16&255;d[e++]=f>>24;var g;switch(w){case 1===a:g=[0,a-1,0];break;case 2===a:g=[1,a-2,0];break;case 3===a:g=[2,a-3,0];break;case 4===a:g=[3,a-4,0];break;case 6>=a:g=[4,a-5,1];break;case 8>=a:g=[5,a-7,1];break;case 12>=a:g=[6,a-9,2];break;case 16>=a:g=[7,a-13,2];break;case 24>=a:g=[8,a-17,3];break;case 32>=a:g=[9,a-25,3];break;case 48>=a:g=[10,a-33,4];break;case 64>=a:g=[11,a-49,4];break;case 96>=a:g=[12,a-65,5];break;case 128>=a:g=[13,a-97,5];break;case 192>=a:g=[14,a-129,6];break;case 256>=a:g=[15,a-193,6];break;case 384>=a:g=[16,a-257,7];break;case 512>=a:g=[17,a-385,7];break;case 768>=a:g=[18,a-513,8];break;case 1024>=a:g=[19,a-769,8];break;case 1536>=a:g=[20,a-1025,9];break;case 2048>=a:g=[21,a-1537,9];break;case 3072>=a:g=[22,a-2049,10];break;case 4096>=a:g=[23,a-3073,10];break;case 6144>=a:g=[24,a-4097,11];break;case 8192>=a:g=[25,a-6145,11];break;case 12288>=a:g=[26,a-8193,12];break;case 16384>=a:g=[27,a-12289,12];break;case 24576>=a:g=[28,a-16385,13];break;case 32768>=a:g=[29,a-24577,13];break;default:throw"invalid distance";}f=g;d[e++]=f[0];d[e++]=f[1];d[e++]=f[2];var k,m;k=0;for(m=d.length;k<m;++k)l[h++]=d[k];s[d[0]]++;x[d[3]]++;q=b.length+c-1;u=null}var e,b,a,g,m,k={},p,t,u,l=C?new Uint16Array(2*d.length):[],h=0,q=0,s=new (C?Uint32Array:Array)(286),x=new (C?Uint32Array:Array)(30),fa=f.i,z;if(!C){for(a=0;285>=a;)s[a++]=0;for(a=0;29>=a;)x[a++]=0}s[256]=1;e=0;for(b=d.length;e<b;++e){a=m=0;for(g=3;a<g&&e+a!==b;++a)m=m<<8|d[e+a];k[m]===n&&(k[m]=[]);p=k[m];if(!(0<q--)){for(;0<p.length&&32768<e-p[0];)p.shift();if(e+3>=b){u&&c(u,-1);a=0;for(g=b-e;a<g;++a)z=d[e+a],l[h++]=z,++s[z];break}0<p.length?(t=Ia(d,e,p),u?u.length<t.length?(z=d[e-1],l[h++]=z,++s[z],c(t,0)):c(u,-1):t.length<fa?u=t:c(t,0)):u?c(u,-1):(z=d[e],l[h++]=z,++s[z])}p.push(e)}l[h++]=256;s[256]++;f.m=s;f.l=x;return C?l.subarray(0,h):l}function Ia(f,d,c){var e,b,a=0,g,m,k,p,t=f.length;m=0;p=c.length;a:for(;m<p;m++){e=c[p-m-1];g=3;if(3<a){for(k=a;3<k;k--)if(f[e+k-1]!==f[d+k-1])continue a;g=a}for(;258>g&&d+g<t&&f[e+g]===f[d+g];)++g;g>a&&(b=e,a=g);if(258===g)break}return new ra(a,d-b)}function pa(f,d){var c=f.length,e=new ia(572),b=new (C?Uint8Array:Array)(c),a,g,m,k,p;if(!C)for(k=0;k<c;k++)b[k]=0;for(k=0;k<c;++k)0<f[k]&&e.push(k,f[k]);a=Array(e.length/2);g=new (C?Uint32Array:Array)(e.length/2);if(1===a.length)return b[e.pop().index]=1,b;k=0;for(p=e.length/2;k<p;++k)a[k]=e.pop(),g[k]=a[k].value;m=Ja(g,g.length,d);k=0;for(p=a.length;k<p;++k)b[a[k].index]=m[k];return b}function Ja(f,d,c){function e(a){var b=k[a][p[a]];b===d?(e(a+1),e(a+1)):--g[b];++p[a]}var b=new (C?Uint16Array:Array)(c),a=new (C?Uint8Array:Array)(c),g=new (C?Uint8Array:Array)(d),m=Array(c),k=Array(c),p=Array(c),t=(1<<c)-d,u=1<<c-1,l,h,q,s,x;b[c-1]=d;for(h=0;h<c;++h)t<u?a[h]=0:(a[h]=1,t-=u),t<<=1,b[c-2-h]=(b[c-1-h]/2|0)+d;b[0]=a[0];m[0]=Array(b[0]);k[0]=Array(b[0]);for(h=1;h<c;++h)b[h]>2*b[h-1]+a[h]&&(b[h]=2*b[h-1]+a[h]),m[h]=Array(b[h]),k[h]=Array(b[h]);for(l=0;l<d;++l)g[l]=c;for(q=0;q<b[c-1];++q)m[c-1][q]=f[q],k[c-1][q]=q;for(l=0;l<c;++l)p[l]=0;1===a[c-1]&&(--g[0],++p[c-1]);for(h=c-2;0<=h;--h){s=l=0;x=p[h+1];for(q=0;q<b[h];q++)s=m[h+1][x]+m[h+1][x+1],s>f[l]?(m[h][q]=s,k[h][q]=d,x+=2):(m[h][q]=f[l],k[h][q]=l,++l);p[h]=0;1===a[h]&&e(h)}return g}function qa(f){var d=new (C?Uint16Array:Array)(f.length),c=[],e=[],b=0,a,g,m,k;a=0;for(g=f.length;a<g;a++)c[f[a]]=(c[f[a]]|0)+1;a=1;for(g=16;a<=g;a++)e[a]=b,b+=c[a]|0,b<<=1;a=0;for(g=f.length;a<g;a++){b=e[f[a]];e[f[a]]+=1;m=d[a]=0;for(k=f[a];m<k;m++)d[a]=d[a]<<1|b&1,b>>>=1}return d};function Ka(f,d){this.input=f;this.a=new (C?Uint8Array:Array)(32768);this.d=V.g;var c={},e;if((d||!(d={}))&&"number"===typeof d.compressionType)this.d=d.compressionType;for(e in d)c[e]=d[e];c.outputBuffer=this.a;this.j=new ka(this.input,c)}var V=na;Ka.prototype.f=function(){var f,d,c,e,b,a,g=0;a=this.a;switch(8){case 8:f=Math.LOG2E*Math.log(32768)-8;break;default:throw Error("invalid compression method");}d=f<<4|8;a[g++]=d;switch(8){case 8:switch(this.d){case V.NONE:e=0;break;case V.h:e=1;break;case V.g:e=2;break;default:throw Error("unsupported compression type");}break;default:throw Error("invalid compression method");}c=e<<6|0;a[g++]=c|31-(256*d+c)%31;var m=this.input;if("string"===typeof m){var k=m.split(""),p,t;p=0;for(t=k.length;p<t;p++)k[p]=(k[p].charCodeAt(0)&255)>>>0;m=k}for(var u=1,l=0,h=m.length,q,s=0;0<h;){q=1024<h?1024:h;h-=q;do u+=m[s++],l+=u;while(--q);u%=65521;l%=65521}b=(l<<16|u)>>>0;this.j.c=g;a=this.j.f();g=a.length;C&&(a=new Uint8Array(a.buffer),a.length<=g+4&&(this.a=new Uint8Array(a.length+4),this.a.set(a),a=this.a),a=a.subarray(0,g+4));a[g++]=b>>24&255;a[g++]=b>>16&255;a[g++]=b>>8&255;a[g++]=b&255;return a};ba("Zlib.Deflate",Ka);ba("Zlib.Deflate.compress",function(f,d){return(new Ka(f,d)).f()});ba("Zlib.Deflate.prototype.compress",Ka.prototype.f);var Ma={NONE:V.NONE,FIXED:V.h,DYNAMIC:V.g},Na,Oa,W,Pa;if(Object.keys)Na=Object.keys(Ma);else for(Oa in Na=[],W=0,Ma)Na[W++]=Oa;W=0;for(Pa=Na.length;W<Pa;++W)Oa=Na[W],ba("Zlib.Deflate.CompressionType."+Oa,Ma[Oa]);}).call(Zlib);
		try { comp = (new Zlib.Zlib.Deflate(bytes.subarray(0, end))).compress(); } catch { throw new Error("Error during compression of output data"); }
		if (comp && comp.length < end) { compressed_length = end; bytes = comp; end = comp.length; }
	}

	// Custom Base62 format for encoding Lua tables into strings (there is no Base62 standard so the way bytes and lengths are encoded is original)
	const Base62_ByteToChar = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'];
	const Base62_CharsForBytes = [ 0, 2, 3, 5 ];
	function Base62_WriteU32(arr, u)
	{
		var tok = Base62_ByteToChar[31 + (u % 31)];
		while (u = (u / 31)|0) tok = Base62_ByteToChar[u % 31] + tok;
		arr.push(tok);
	}
	function Base62_WriteData(arr, data, datalen)
	{
		for (var chksum = 0, i = 0; datalen > 0; datalen -= 4, i += 4)
		{
			var nchars = (datalen > 3 ? 6 : Base62_CharsForBytes[datalen]), tok = '';
			var bits = (data[i+0] | (data[i+1] << 8) | (data[i+2] << 16)) + ((data[i+3]|0) * 0x1000000); // keep unsigned
			chksum = ((chksum + bits) % 4294967296);
			for (var j = 0; j != nchars; j++, bits = (bits / 62)|0)
				tok = Base62_ByteToChar[bits % 62] + tok;
			arr.push(tok);
		}
		arr.push(Base62_ByteToChar[chksum % 62]);
	}

	// Fill token array and return joined result string
	var arr = ["DS" + (type ? type.substr(0,1) : '?')];
	Base62_WriteU32(arr, compressed_length);
	Base62_WriteData(arr, bytes, end);
	return arr.join('');
}

function copyToClipboard(text) {
  // Create a textarea element
  var textarea = document.createElement('textarea');
  // Set the value of the textarea to the text you want to copy
  textarea.value = text;
  // Append the textarea to the DOM
  document.body.appendChild(textarea);
  // Select the text within the textarea
  textarea.select();
  // Copy the selected text to the clipboard
  document.execCommand('copy');
  // Remove the textarea from the DOM
  document.body.removeChild(textarea);
}
var t = ObjectToDesyncedString(jsonstring, 'C')
copyToClipboard(t)


<IPython.core.display.Javascript object>

In [503]:
source_code='''
for A, B in LoopSignalMatch(P3):
    P4=C
    
while CheckBattery(GetSelf()):
    B=B+1
    A=1+2+3'''
source_code='''
A=B+1
'''
tree = ast.parse(source_code, type_comments=False)
tree = replace_binops_with_functions(tree)
tree = transform_nested_calls(tree)
tree = label_frames(tree)
flow_control(tree)
frames = walk(tree)
jsonstring = json.dumps(frames)
print(jsonstring)

{"0": {"op": "add", "next": -1, "0": "B", "1": {"num": 1}, "2": "A"}}


In [508]:

test=123

In [509]:
%%javascript

var pythonVariable = IPython.notebook.kernel.execute('python_variable').result;


<IPython.core.display.Javascript object>